In [ ]:
# 最高ウェイト表現のウェイトを最高ウェイトとの差で表す関数。
def root_difference_multiplicities(character_ring, highest_weight) -> dict:
    weight_muliplicities = character_ring(highest_weight).weight_multiplicities()
    A = matrix([ vector(sr) for sr in character_ring.simple_roots() ]).transpose()

    result ={}

    for k, v in weight_muliplicities.items():
        Y = vector(k) - vector(highest_weight)
        result[tuple(A.solve_right(Y))] = v
    
    return result

In [ ]:
# Parabolic subgroupを表すクラス
class ParabolicSubgroup:
    def __init__(self, G, L, crossed_out_nodes) -> None:
        self.G = G
        self.L = L
        self.crossed_out_nodes = crossed_out_nodes
        self.R_G = WeylCharacterRing(self.G)
        self.R_L = WeylCharacterRing(self.L)

    def __repr__(self) -> str:
        return f'the parabolic subgroup of {self.G} with crossed-out nodes {self.crossed_out_nodes}'

    def dynkin_diagram(self):
        return self.G.marked_nodes(self.crossed_out_nodes).dynkin_diagram()

    def ambient_space(self):
        return self.R_G.space()
    
    def ambient_space_dimension(self):
        return self.R_G.space().dimension()

    # Sagemathのライブラリでは1-indexを用いるが、ここでは簡単のためとりあえず0-indexで表す
    def simple_roots(self):
        return [ self.R_G.simple_roots()[i] for i in set(1..self.G.rank()) - set(self.crossed_out_nodes)]

    def positive_roots(self):
        L = self.ambient_space()

        # positive rootからcutoutされるsimple rootを引き, positiveでなければuncrossed nodeから生成されると判定
        roots = [ pr for pr in self.G.root_system().root_lattice().positive_roots()]
        for i in self.crossed_out_nodes:
            roots = [ pr for pr in roots if not (pr - (self.G.root_system().root_lattice().simple_roots())[i]).is_positive_root()]

        # ambient spaceの元に変換
        return [ L(pr) for pr in roots ]

    # 引数の`weight`は基本ウェイトを基底にして表示したもの
    def weight_muliplicities(self, weight) -> dict:
        # GとLのディンキン図の頂点のずれを補正する関数
        def correct_index(index: int) -> int:
            for i in range(len(self.crossed_out_nodes)):
                if index + i < self.crossed_out_nodes[i]:
                    return index + i
            return index + len(self.crossed_out_nodes)

        fws_L = [fw for fw in self.R_L.fundamental_weights()] # conversion from 1-index to 0-index
        weight_for_L = [ weight[i - 1] for i in set(1..len(weight))-set(self.crossed_out_nodes)] 
        weight_for_L = sum( weight_for_L[i] * fws_L[i] for i in range(self.L.rank()))

        fws_G = [fw for fw in self.R_G.fundamental_weights()]
        weight_for_G = sum( weight[i] * fws_G[i] for i in range(self.G.rank()))

        mul_set = root_difference_multiplicities(self.R_L, weight_for_L)

        result = {}
        for k, v in mul_set.items():
            w = weight_for_G \
                + sum(k[i - 1] * self.R_G.simple_roots()[correct_index(i)] 
                    for i in (1..self.L.rank()))
            result[w] = v

        return result
    
    def weyl_dimension(self, weight) -> int:
        return self.ambient_space().weyl_dimension(sum(weight[i - 1] * self.R_G.fundamental_weights()[i] for i in (1..self.G.rank())))

In [ ]:
P = ParabolicSubgroup(CartanType('A3'), CartanType('A2'), [1])
P

In [ ]:
P.dynkin_diagram()

In [ ]:
P.ambient_space_dimension()

In [ ]:
P.simple_roots()

In [ ]:
P.positive_roots()

In [ ]:
P.weight_muliplicities((1, 2, 0))

In [ ]:
from abc import ABC, abstractmethod

class IVectorBundle(ABC):
    pass

class IVariety(ABC):
    @abstractmethod
    def dimension(self) -> int:
        pass

    @abstractmethod
    def tangent_bundle(self) -> IVectorBundle:
        pass

    def cotangent_bundle(self) -> IVectorBundle:
        return self.tangent_bundle().dual()

    @abstractmethod
    def integration(self, f) -> int:
        pass

    def chern_classes(self) -> list:
        return self.tangent_bundle().chern_classes()

    def todd_classes(self) -> list:
        return self.tangent_bundle().todd_classes()

In [ ]:
import re
singular.lib("chern.lib")


class IVectorBundle(ABC):
    @abstractmethod
    def base(self) -> IVariety:
        pass

    @abstractmethod
    def rank(self) -> int:
        pass

    @abstractmethod
    def chern_classes(self) -> list:
        pass

    # 演算子のオーバーロード
    def __add__(self, other) -> IVectorBundle:
        return direct_sum(self, other)

    def __mul__(self, other) -> IVectorBundle:
        return tensor_product(self, other)

    def chern_character(self) -> list:
        len_cc = len(self.chern_classes()) - 1

        ring_for_ch = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))
        
        # Using Singular, calclate universal formula of chern character
        singular.lib('chern.lib')
        r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
        l = singular.list(f'c(1..{len_cc})')
        ch_str_list = singular.chAll(l, self.base().dimension()).sage_structured_str_list()
        chern_character = [ ring_for_ch(self.rank()) ] + [ ring_for_ch(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_str_list ]

        chern_classes = self.chern_classes()[1:]

        return [ chern_character[i](chern_classes) for i in (0..self.base().dimension()) ]

    def todd_classes(self) -> list:
        len_cc = len(self.chern_classes()) - 1

        ring_for_td = PolynomialRing(QQ, [ f'c{i}_M' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))

        # Using Singular, calculate universal formula of Todd classes
        r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
        l = singular.list(f'c(1..{len_cc})')
        todd_str_list = singular.todd(l).sage_structured_str_list()
        todd_classes = [ ring_for_td(1) ] + [ ring_for_td(re.sub(r'c\(([0-9]+)\)', r'c\1_M', s)) for s in todd_str_list ]

        chern_classes = self.chern_classes()[1:]

        return [ todd_classes[i](chern_classes) for i in (0..self.base().dimension()) ]

    def dual(self) -> IVectorBundle:
        vector_bundle = self

        class VB(IVectorBundle):
            def rank(self) -> int:
                return vector_bundle.rank()
            
            def base(self) -> IVariety:
                return vector_bundle.base()

            def chern_classes(self) -> list:
                chern_classes = vector_bundle.chern_classes()
                return [ (-1)^i * chern_classes[i] for i in range(len(chern_classes))]

            def __repr__(self) -> str:
                return f'the dual vector bundle of {vector_bundle}'

        return VB()

In [ ]:
def direct_sum(vector_bundle1: IVectorBundle, vector_bundle2: IVectorBundle):
    if vector_bundle1.base() != vector_bundle2.base():
        raise TypeError('Not match bases of vector bundles')
    
    
    rank = vector_bundle1.rank() + vector_bundle2.rank()
    base = vector_bundle1.base()

    cc = sum(c1 for c1 in vector_bundle1.chern_classes()) * sum(c2 for c2 in vector_bundle2.chern_classes())
    chern_classes = [
        homogeneous_part(cc, i)
        for i in (0..vector_bundle1.base().dimension())
    ]

    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base

        def chern_classes(self) -> list:
            return chern_classes

        def __repr__(self) -> str:
            return f'the direct sum of {vector_bundle1} and {vector_bundle2}'

    return VB()

In [ ]:
def tensor_product(vector_bundle1: IVectorBundle, vector_bundle2: IVectorBundle):
    if vector_bundle1.base() != vector_bundle2.base():
        raise TypeError('Not match bases of vector bundles')
    
    rank1 = vector_bundle1.rank()
    rank2 = vector_bundle2.rank()

    len_cc1 = len(vector_bundle1.chern_classes()) - 1
    len_cc2 = len(vector_bundle2.chern_classes()) - 1
    
    ring_for_Es = PolynomialRing(QQ, [ f'c{i}_E1' for i in (1..len_cc1) ] + [ f'c{i}_E2' for i in (1..len_cc2) ])
    

    r = singular.ring(0, f'(c(1..{len_cc1}), C(1..{len_cc2}))', 'dp')
    l1 = singular.list(f'c(1..{len_cc1})')
    l2 = singular.list(f'C(1..{len_cc2})')

    ch_str_list = singular.chProd(rank1, l1, rank2, l2).sage_structured_str_list()
    ch_prod = [ re.sub(r'c\(([0-9]+)\)', r'c\1_E1', s) for s in ch_str_list[:vector_bundle1.base().dimension()] ]
    ch_prod = [ ring_for_Es(re.sub(r'C\(([0-9]+)\)', r'c\1_E2', s)) for s in ch_prod ]


    cc = [ 1 ] + [ cp(vector_bundle1.chern_classes()[1:] + vector_bundle2.chern_classes()[1:]) for cp in ch_prod ]
    

    rank = vector_bundle1.rank() * vector_bundle2.rank()
    base = vector_bundle1.base()


    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base

        def chern_classes(self) -> list:
            return cc

        def __repr__(self) -> str:
            return f'the tensor product of {vector_bundle1} and {vector_bundle2}'

    return VB()

In [ ]:
# `degree`次部分を取り出す関数
homogeneous_part = lambda F, degree: sum( c*m for c, m in F if m.total_degree() == degree )

In [ ]:
def chern_number(variety: IVariety, degrees: list) -> int:
    if sum(d for d in degrees) != variety.dimension():
        return 0
    else:
        chern_classes = variety.chern_classes()
        return variety.integration(
            prod([chern_classes[d] for d in degrees])
        )

In [ ]:
class FlagVariety(IVariety):
    def __init__(self, parabolic_subgroup) -> None:
        self.parabolic_subgroup = parabolic_subgroup

        # コホモロジー環を含む環
        self.ring = PolynomialRing(QQ, 'x', parabolic_subgroup.ambient_space_dimension()) 
        self.x = self.ring.gens()

        self.tangent_weights = [
            sum( r[l] * self.x[l] for l in range(parabolic_subgroup.ambient_space_dimension())) 
            for r in set(parabolic_subgroup.R_G.positive_roots())-set(parabolic_subgroup.positive_roots())
        ]
        self.dim = len(self.tangent_weights)

    def __repr__(self) -> str:
        return f'a flag_variety associated to {self.parabolic_subgroup}'

    def dimension(self) -> int:
        return self.dim
    
    def tangent_bundle(self):
        tangent_weights = { w: 1 for w in set(self.parabolic_subgroup.R_G.positive_roots())-set(self.parabolic_subgroup.positive_roots()) }
        return EquivariantVectorBundle(self, tangent_weights)


    # 次数ごとのchern類
    def chern_classes(self):
        return [ 
            homogeneous_part(prod(1+x for x in self.tangent_weights), i)
            for i in (0..self.dim) 
        ]

    def numerical_integration_by_localization(self, f):
        random_x = [RealField(1000)(random()) for i in range(self.parabolic_subgroup.ambient_space_dimension())]
        orbit_of_random_x = [(w.inverse()*vector(RealField(1000),random_x)).list() for w in WeylGroup(self.parabolic_subgroup.G)]
        top_of_f = homogeneous_part(f, self.dim)
        denominator_in_localization = prod(self.tangent_weights)
        if top_of_f == 0:
            return 0
        else:
            return sum([top_of_f(x)/denominator_in_localization(x) for x in orbit_of_random_x]).round() / len(WeylGroup(self.parabolic_subgroup.L))

    def integration(self, f) -> int:
        return self.numerical_integration_by_localization(f)
     

In [ ]:
class EquivariantVectorBundle(IVectorBundle):
    def __init__(self, flag_variety, weight_muliplicities) -> None:
        self.flag_variety = flag_variety
        self.weight_muliplicities = weight_muliplicities
        self.rk = sum(v for v in self.weight_muliplicities.values())

    def __repr__(self) -> str:
        return f'an equivariant vector bundle on {self.flag_variety} associated to {self.weight_muliplicities}'

    def rank(self) -> int:
        return self.rk

    def base(self) -> FlagVariety:
        return self.flag_variety

    def chern_classes(self):
        def class_from_weight(weight):
            return sum(weight[i] * self.flag_variety.x[i] for i in range(self.flag_variety.parabolic_subgroup.ambient_space_dimension()))
        
        cc = prod((1 + class_from_weight(vector(w)))^i for w, i in self.weight_muliplicities.items())

        return [
            homogeneous_part(cc, i)
            for i in (0..self.flag_variety.dim)
        ]

class IrreducibleEquivariantVectorBundle(EquivariantVectorBundle):
    def __init__(self, flag_variety, weight) -> None:
        self.weight = weight
        super().__init__(flag_variety, flag_variety.parabolic_subgroup.weight_muliplicities(weight))

    def __repr__(self) -> str:
        return f'an equivariant vector bundle on {self.flag_variety} associated to {self.weight}'

- Test of FlagVariety

In [ ]:
X = FlagVariety(P)
X

In [ ]:
X.dimension()

In [ ]:
X.tangent_weights

In [ ]:
X.chern_classes()

In [ ]:
X.todd_classes()

In [ ]:
X.tangent_bundle()

In [ ]:
X.cotangent_bundle().chern_classes()

In [ ]:
[ f"{p} -> {chern_number(X, p)}"for p in Partitions(X.dimension())]

- Test for EquivariantVectorBundle

In [ ]:
w = (4, 0, 0)
E = IrreducibleEquivariantVectorBundle(X, w)
E

In [ ]:
E.base()

In [ ]:
E.rank()

In [ ]:
E.chern_classes()

In [ ]:
E.chern_character()

In [ ]:
E.todd_classes()

In [ ]:
class CompleteIntersection(IVariety):
    def __init__(self, flag_variety, vector_bundle) -> None:
        self.flag_variety = flag_variety
        self.vector_bundle = vector_bundle
        self.dim = self.flag_variety.dimension() - self.vector_bundle.rank()

    def __repr__(self) -> str:
        return f'a complete intersection of {self.flag_variety} and {self.vector_bundle}'

    def dimension(self) -> int:
        return self.dim
    
    def tangent_bundle(self):
        ci = self
        class VB(IVectorBundle):
            def base(self) -> IVariety:
                return ci
            def rank(self) -> int:
                return ci.dim
            def chern_classes(self) -> list:
                return ci.chern_classes()
        
        return VB()

    def chern_classes(self):
        def class_from_weight(weight):
            return sum(weight[i] * self.flag_variety.x[i] for i in range(self.flag_variety.parabolic_subgroup.ambient_space_dimension()))
        
        def geometric_sequence(n, x):
            return sum( x^i for i in (0..n))
        
        cc = prod(1+x for x in self.flag_variety.tangent_weights) \
            * \
            prod(geometric_sequence(self.dim, -class_from_weight(vector(w)))^i 
                for w, i in self.vector_bundle.weight_muliplicities.items()
            )
        
        return [
            homogeneous_part(cc, i)
            for i in (0..self.dim)
        ]

    def numerical_integration_by_localization(self, f):
        top_of_f = homogeneous_part(f, self.dim)
        c_top    = self.vector_bundle.chern_classes()[self.vector_bundle.rank()] if self.dim >= 0 else 0
        return self.flag_variety.numerical_integration_by_localization(top_of_f * c_top)

    def integration(self, f) -> int:
        return self.numerical_integration_by_localization(f)


In [ ]:
# K3
Y = CompleteIntersection(X, E) 
Y.dimension()

In [ ]:
Y.chern_classes()

In [ ]:
Y.tangent_bundle().chern_classes()

In [ ]:
Y.cotangent_bundle().chern_classes()

In [ ]:
Y.integration(Y.chern_classes()[Y.dimension()])

In [ ]:
[ f"{p} -> {chern_number(Y, p)}"for p in Partitions(Y.dimension())]

Euler Characteristic
$$
\chi (X, E) = \int_X ch(E) td(X)
$$

In [ ]:
def euler_characteristic(variety: IVariety, vector_bundle: IVectorBundle) -> int:
    chern_character = vector_bundle.chern_character()
    todd_classes    = variety.todd_classes()
    
    return variety.integration( 
        sum(
            chern_character[i]
            * todd_classes[variety.dimension() - i]
            for i in (0..variety.dimension())
        )
    )

In [ ]:
euler_characteristic(X, E)

In [ ]:
P.weyl_dimension(w)

- Tests for 
    - dual bundle
    - direct sum
    - tensor product
    - Symmetric products
    - Wedge products

In [ ]:
E = IrreducibleEquivariantVectorBundle(X, (1, 1, 0))
E.rank()

In [ ]:
dual_of_E = E.dual()
dual_of_E

In [ ]:
dual_of_E.rank()

In [ ]:
E.chern_classes()

In [ ]:
dual_of_E.chern_classes()

In [ ]:
L1 = IrreducibleEquivariantVectorBundle(X, (2, 0, 0))
L2 = IrreducibleEquivariantVectorBundle(X, (5, 0, 0))
L3 = IrreducibleEquivariantVectorBundle(X, (7, 0, 0))

In [ ]:
direct_sum(L1, L2)

In [ ]:
direct_sum(L1, L2).rank()

In [ ]:
direct_sum(L1, L2).chern_classes()

In [ ]:
(L1 + L2 + L3).chern_classes()

In [ ]:
E_otimes_L1 = tensor_product(E, L1)
E_otimes_L1

In [ ]:
(L1 * L2 * L3).chern_classes()

In [ ]:
def symmetric_product(vector_bundle: IVectorBundle, k: int) -> IVectorBundle:
    chern_classes = vector_bundle.chern_classes()[1:]
    rank          = vector_bundle.rank()
    len_cc        = len(chern_classes)

    ring_for_E = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))

    r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
    l = singular.list(f'c(1..{len_cc})')
    ch_symm_str_list = singular.chSymm(k, rank, l).sage_structured_str_list()
    ch_symm = [ ring_for_E(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_symm_str_list[1][:vector_bundle.base().dimension()] ]

    rank = int(ch_symm_str_list[0])
    base = vector_bundle.base()
    cc = [ 1 ] + [ cs(chern_classes) for cs in ch_symm ]

    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base
        
        def chern_classes(self) -> list:
            return cc

        def __repr__(self) -> str:
            return f'the {k}-th symmetric product of {vector_bundle}'

    return VB()

In [ ]:
S3_of_E = symmetric_product(E, 3)
S3_of_E.rank()

In [ ]:
S3_of_E.chern_classes()

In [ ]:
def wedge_product(vector_bundle: IVectorBundle, k: int) -> IVectorBundle:
    chern_classes = vector_bundle.chern_classes()[1:]
    rank          = vector_bundle.rank()
    len_cc        = len(chern_classes)

    ring_for_E = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..len_cc) ], order=TermOrder('wdeglex', tuple(1..len_cc)))

    r = singular.ring(0, f'(c(1..{len_cc}))', 'dp')
    l = singular.list(f'c(1..{len_cc})')
    ch_wedge_str_list = singular.chWedge(k, rank, l).sage_structured_str_list()
    ch_wedge = [ ring_for_E(re.sub(r'c\(([0-9]+)\)', r'c\1_E', s)) for s in ch_wedge_str_list[1][:vector_bundle.base().dimension()] ]

    rank = int(ch_wedge_str_list[0])
    base = vector_bundle.base()
    cc   = [ 1 ] + [ cs(chern_classes) for cs in ch_wedge ]

    class VB(IVectorBundle):
        def rank(self) -> int:
            return rank
        
        def base(self) -> IVariety:
            return base
        
        def chern_classes(self) -> list:
            return cc

        def __repr__(self) -> str:
            return f'the {k}-th wedge product of {vector_bundle}'

    return VB()

In [ ]:
L2_of_E = wedge_product(E, 2)
L2_of_E

In [ ]:
L2_of_E.rank()

In [ ]:
L2_of_E.chern_classes()

In [ ]:
wedge_product(E, 3).rank()

In [ ]:
wedge_product(E, 5).rank()

$\chi_y$-Genus
$$
\chi_y(X) = \int_X ch(\Lambda_y T^{\vee}_X) td(X)
$$

In [ ]:
Q_y = QQ['y']
y = Q_y.gen()

def chi_y(variety: IVariety):
    cotangent_bundle = variety.cotangent_bundle()
    todd_classes = variety.todd_classes()

    coeff = [ 0 ] * (variety.dimension() + 1)

    for k in (0..variety.dimension()):
        chern_character = wedge_product(cotangent_bundle, k).chern_character()
        coeff[k] = variety.integration( 
            sum(
                chern_character[i]
                * todd_classes[variety.dimension() - i]
                for i in (0..variety.dimension())
            )
        )

    return sum(coeff[i] * y^i for i in (0..variety.dimension()))

In [ ]:
chi_y(Y)

In [ ]:
chi_y(Y)(-1) # オイラー標数と同じ

- Elliptic Genus

$$
E_{q, y} = y^{-d/2} \bigotimes_{n \geq 1} 
\Lambda_{-yq^{n-1}} T^\vee_X 
\otimes 
\Lambda_{-y^{-1}q^n} T_X 
\otimes
Sym_{q^n} T^\vee_X
\otimes
Sym_{q^n} T_X
$$

$$
Ell_{q, y} = \int_X ch(E_{q, y})td(X)
$$

ここではプログラム処理上の理由で、$y^{d/2}Ell_{q,y}$を楕円種数と呼ぶことにする

In [ ]:
import itertools

# 楕円種数のq^n*y^m次部分
def elliptic_genus_coefficient(variety: IVariety, n: int, m: int):
    cb = variety.cotangent_bundle()
    tb = variety.tangent_bundle()
    todd_classes = variety.todd_classes()

    wcb = [
        wedge_product(cb, i)
        for i in (0..max(variety.dimension(), n + 1))
    ]


    wtb = [
        wedge_product(tb, j)
        for j in (0..max(variety.dimension(), n))
    ]

    scb = [
        symmetric_product(cb, k)
        for k in (0..n)
    ]

    stb = [
        wedge_product(tb, l)
        for l in (0..n)
    ]

    indices = itertools.product(
        (0..max(variety.dimension(), n + 1)), 
        (0..max(variety.dimension(), n)), 
        (0..n), 
        (0..n)
    )
    indices = itertools.product(indices, repeat=n + 1)

    result = 0

    for x in indices:
        if sum(x[i][0] - x[i][1] for i in range(n + 1)) == m \
            and sum(x[i - 1][0] * (i - 1) + x[i - 1][1] * i + x[i - 1][2] * i + x[i - 1][3] * i for i in (1..n + 1)) == n:

            vb = prod(wcb[x[i][0]] * wtb[x[i][1]] * scb[x[i][2]] * stb[x[i][3]] for i in range(n + 1))
            chern_character = vb.chern_character()


            result += variety.integration(
                sum(
                    chern_character[i]
                    * todd_classes[variety.dimension() - i]
                    for i in (0..variety.dimension())
                )
            ) * (-1)^m

    return result

In [ ]:
Q_qy.<q, y> = LaurentPolynomialRing(QQ)
# q, y = Q_qy.gens()

def elliptic_genus(variety: IVariety, cutoff_q: int, cutoff_y: int):
    result = 0
    for n in (0..cutoff_q):
        for m in (-cutoff_y..cutoff_y):
            result += elliptic_genus_coefficient(variety, n, m) * q^n * y^m

    return result

In [ ]:
elliptic_genus(Y, 0, 3)

In [ ]:
elliptic_genus(Y, 1, 3) # 本当はこれにy^(-1)を掛けたものが正しい楕円種数

In [ ]:
class DummyCalabiYau(IVariety):
    def __init__(self, dim: int) -> None:
        self.dim = dim
        self.ring = PolynomialRing(QQ, [f"c{i}" for i in (2..dim)], order=TermOrder('wdeglex', tuple(2..dim)))

    def __repr__(self) -> str:
        return f"a dummy variety of dimension {self.dim}"

    def dimension(self) -> int:
        return self.dim

    def tangent_bundle(self) -> IVectorBundle:
        variety = self
        class VB(IVectorBundle):
            def base(self) -> IVariety:
                return variety
            
            def rank(self) -> int:
                return variety.dimension()
            
            def chern_classes(self) -> list:
                cs = variety.ring.gens()
                return [1, 0] + list(cs)
        
        return VB()

    def integration(self, f) -> int:
        return homogeneous_part(f, self.dim)

    def chi_y(self):
        dummyring = PolynomialRing(self.ring, 'y')
        y = dummyring.gen()
        
        cotangent_bundle = self.cotangent_bundle()
        todd_classes = self.todd_classes()

        coeff = [ 0 ] * (self.dimension() + 1)

        for k in (0..self.dimension()):
            chern_character = wedge_product(cotangent_bundle, k).chern_character()
            coeff[k] = sum(
                    chern_character[i]
                    * todd_classes[self.dimension() - i]
                    for i in (0..self.dimension())
                )
            

        return sum(coeff[i] * y^i for i in (0..self.dimension()))

In [140]:
dummy = DummyCalabiYau(17)
dummy

a dummy variety of dimension 17

In [ ]:
dummy.chern_classes()

In [ ]:
dummy.cotangent_bundle().chern_character()

In [ ]:
dummy.chi_y()

======= Quintic 3-fold =======

In [ ]:
P = ParabolicSubgroup(CartanType('A4'), CartanType('A3'), [1])
P.dynkin_diagram()

In [ ]:
X = FlagVariety(P)
E = IrreducibleEquivariantVectorBundle(X, (5, 0, 0, 0, 0))
quintic = CompleteIntersection(X, E)

In [ ]:
quintic.chern_classes()

In [ ]:
chi_y(quintic)

In [ ]:
elliptic_genus(quintic, 2, 3)

In [ ]:
[ f"{p} -> {chern_number(quintic, p)}"for p in Partitions(quintic.dimension())]

======= G2 cases =======

In [ ]:
G2 = CartanType('G2')
G2.dynkin_diagram()

In [ ]:
P1 = ParabolicSubgroup(G2, CartanType('A1'), [1])
P2 = ParabolicSubgroup(G2, CartanType('A1'), [2])

In [ ]:
P1.dynkin_diagram()

In [ ]:
P2.dynkin_diagram()

In [ ]:
X1 = FlagVariety(P1)
X2 = FlagVariety(P2)

In [ ]:
w = (1, 1)
E1 = IrreducibleEquivariantVectorBundle(X1, w)
E2 = IrreducibleEquivariantVectorBundle(X2, w)

In [ ]:
Y1 = CompleteIntersection(X1, E1)
Y2 = CompleteIntersection(X2, E2)

In [ ]:
Y1.chern_classes()

In [ ]:
Y2.chern_classes()

In [ ]:
Y1.integration(Y1.chern_classes()[Y1.dimension()])

In [ ]:
Y2.integration(Y2.chern_classes()[Y2.dimension()])

In [ ]:
elliptic_genus(Y1, 2, 3)

In [ ]:
elliptic_genus(Y2, 2, 3)

======= F4 cases =======

In [ ]:
G = CartanType('F4')
G.dynkin_diagram()

In [ ]:
P1 = ParabolicSubgroup(G, CartanType('A1', 'A2'), [2])
P1

In [ ]:
P1.dynkin_diagram()

In [ ]:
X1 = FlagVariety(P1)
X1

In [ ]:
w1 = (0, 1, 1, 0)
E1 = IrreducibleEquivariantVectorBundle(X1, w1)
E1

In [ ]:
E1.rank()

In [ ]:
# euler_characteristic(X1, E1)

In [ ]:
P1.weyl_dimension(w1)

In [ ]:
w2 = (5, 0, 0, 0)
E2 = IrreducibleEquivariantVectorBundle(X1, w2)
E2.rank()

In [ ]:
# euler_characteristic(X1, E2)

In [ ]:
P1.weyl_dimension(w2)

In [ ]:
P2 = ParabolicSubgroup(G, CartanType('A2', 'A1'), [3])
P2

In [ ]:
P2.dynkin_diagram()

In [ ]:
X2 = FlagVariety(P2)
X2

In [ ]:
w3 = (0, 1, 1, 0)
E3 = IrreducibleEquivariantVectorBundle(X2, w3)
E3

In [ ]:
euler_characteristic(X2, E3)

In [ ]:
P2.weyl_dimension(w3)

In [ ]:
w4 = (2, 1, 0, 0)
E4 = IrreducibleEquivariantVectorBundle(X2, w4)
E4.rank()

In [ ]:
euler_characteristic(X2, E4)

In [ ]:
P2.weyl_dimension(w4)

In [ ]:
Y1 = CompleteIntersection(X1, E1)
Y2 = CompleteIntersection(X2, E3)

In [ ]:
# Y1.chern_classes()

In [ ]:
# Y2.chern_classes()

In [ ]:
# e1 = elliptic_genus(Y1, 1, 2)

In [ ]:
# e2 = elliptic_genus(Y2, 1, 2)

In [ ]:
# e1 == e2

In [ ]:
chern_number(Y1, [17])

In [ ]:
chern_number(Y2, [17])

In [ ]:
chern_number(Y1, [2] * 7 + [3])

In [ ]:
chern_number(Y2, [2] * 7 + [3])

In [ ]:
class DummyVariety(IVariety):
    def __init__(self, dim: int) -> None:
        self.dim = dim
        self.ring = PolynomialRing(QQ, [f"c{i}" for i in (1..dim)], order=TermOrder('wdeglex', tuple(1..dim)))

    def __repr__(self) -> str:
        return f"a dummy variety of dimension {self.dim}"

    def dimension(self) -> int:
        return self.dim

    def tangent_bundle(self) -> IVectorBundle:
        variety = self
        class VB(IVectorBundle):
            def base(self) -> IVariety:
                return variety
            
            def rank(self) -> int:
                return variety.dimension()
            
            def chern_classes(self) -> list:
                cs = variety.ring.gens()
                return [1] + list(cs)
        
        return VB()

    def integration(self, f) -> int:
        return homogeneous_part(f, self.dim)

In [ ]:
d = DummyVariety(3)
d

In [ ]:
d.chern_classes()

In [ ]:
d.tangent_bundle().chern_character()

In [ ]:
[ f"{p} -> {chern_number(d, p)}"for p in Partitions(d.dimension())]